# Data Analysis

## Solutions object
Obtain the Hidden Sector from: https://zenodo.org/record/5526707#.YzwIANJBzeQ and add the missing the column `Hidden Sector` of the following data scheme

![img](./squeme.svg)

### Example of entry

In [1]:
from IPython.display import JSON 
import pandas as pd

In [2]:
JSON({'A':1})

<IPython.core.display.JSON object>

In [3]:
JSON(([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )
    )


<IPython.core.display.JSON object>

In [4]:
import pandas as pd

In [5]:
pd.DataFrame([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )

,l,k,solution,gcd,n,hidden
0,"[1, 3]",[2],"[1, 2, -3, 4, 5]",1,5,"[{'ψ': [(1, 5), (2, 4), (-3,)], 'S': 6}]"


### Read the full dataset

In [6]:
df=pd.read_json('https://zenodo.org/record/5526707/files/solutions.json?download=1')

In [7]:
FULL=True
if FULL:
    #From https://zenodo.org/record/7380817/files/solutions.json.gz?download=1
    df=pd.read_json('solutions.json.gz')
    df=df[df['solution'].str[-1].abs()<=20].reset_index(drop=True)

In [8]:
df[df['n']==5].shape

(3, 5)

In [9]:
df.shape

(96153, 5)

The `solutions` column, was generated with an ordering in absolute value, like the following

In [10]:
sorted([-2,1,-8,5],key=abs)

[1, -2, 5, -8]

This allows to take the maximum directly from the last element

In [11]:
df['solution'].str[-1]

0         3
1         6
2         4
3        20
4        12
         ..
96148    19
96149    20
96150   -20
96151   -20
96152   -20
Name: solution, Length: 96153, dtype: int64

In [12]:
df['zmax']=df['solution'].str[-1].abs()
df=df.sort_values(['n','zmax']).reset_index(drop=True)

#How many solutions with n=12 and z_max <= 20?
df[ (df['n']==12) & (df['zmax']<=20) ].shape

(65910, 6)

In [13]:
df[df['n']==5]

,l,k,solution,gcd,n,zmax
0,[-2],"[2, 3]","[1, 5, -7, -8, 9]",4,5,9
1,[3],"[-3, -2]","[2, 4, -7, -9, 10]",18,5,10
2,[-2],"[3, 1]","[1, 14, -17, -18, 20]",6,5,20


### Add the new column with the list of hidden sectors

After refactoring...

In [14]:
import itertools
import math as mh

def remove_double_intersection_pairs(msvp,l):
    '''
    Example:
        msvp = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        # Remove (1,6) because is fully contained 
        # in the intersections with (_1_,-8) and (_6_,-13)
        returns: 
        [(3,4),(6,-13),(-7,14),(1,-8)]    
    '''
    for p in msvp.copy():
        tmp=msvp.copy()
        tmp.remove(p)
        fltmp=set([x for sublist in tmp for x in sublist]) 
        if l.count(p[0])==1 and l.count(p[1])==1 and set(p).issubset(fltmp):
            msvp=tmp.copy()
    return msvp
assert len(remove_double_intersection_pairs([(-10, 11), (-12, 13), (11, -12)],
                                             [-10, 11, -12, 13]))==2
assert len(remove_double_intersection_pairs([(4,4),(4,-12)],[4,-12]))==1

def get_real_massless(l,msvp):
    '''
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        msvp = [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        [] # lp    
    '''
    lp=l.copy()
    for x,y in msvp:
        #print(lp,msvp,x,y)
        if lp and x in lp and y in lp:
            minxy=min(lp.count(x),lp.count(y))
            if x!=y:
                [(lp.remove(x),lp.remove(y)) for ii in range(minxy)]
            else:
                [ lp.remove(x) for ii in range(lp.count(x)) ] # abs(2*x)==s
        else: # not lp
            break
    return lp

def get_hidden_fermions(l,s,ps=[]):
    '''
    For a given scalar Abelian charge `s`, build the 
    fermion non-zero determinant mass matrix from their
    Abelian charges in `l`, and returns the input `s` and 
    the non-zero mass entries.
    Returns an empty dictionary otherwise.
    
    The list of combinations on pairs of `l` can be given in `ps`,
    otherwise it is internally calculated.
    
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        s = 7
        ps = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        # If necessary temove [1,6] because have is fully contained 
        # in the intersections with others
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        {'s':7, '\psi':[(3,4),(6,-13),(1,6),(-7,14),(1,-8)]}

    '''
    sltn={}
    if not ps:
        ps = list(set(itertools.combinations_with_replacement(l,2)))
    msv = [p for p in ps if abs(sum(p))==s]
    lmsv=[x for sublist in msv for x in sublist]
    massives=set(lmsv)        
    massless=set(l).difference(massives)
    if not massless:
        # ========= Quality check =======================
        # Check that all the elements of l can be removed
        for INTERSECTIONS in [False,True]:
            msvp=msv.copy()
            lp=l.copy()
            if INTERSECTIONS:
                msvp = remove_double_intersection_pairs(msvp,l)
            for msvp in itertools.permutations(msvp.copy()):
                msvp=list(msvp)
                lp=get_real_massless(l,msvp)
                if not lp:
                    break # found real massless
            if not lp: # double break for INTERSECTIONS
                break # found real massless
                    
        
        if not lp:
            sltn['S']=s
            sltn['ψ']=msv
        return sltn
assert get_hidden_fermions([2, 4, 4, 5, 5, 7],9).get('S')==9    


def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        sltn=get_hidden_fermions(l,s,ps)
        if sltn:
            sltns.append(sltn)
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1 # Ana test
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8
assert get_hidden_sector([3, 3, -4, -4, -4, 5, -6, 7, -8, 9, 10, -11])[0].get('S')==1

In [15]:
import itertools
import math as mh

assert get_hidden_fermions([2, 4, 4, 5, 5, 7],9).get('S')==9    

def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        sltn=get_hidden_fermions(l,s,ps)
        if sltn:
            sltns.append(sltn)
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1 # Ana test
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8
assert get_hidden_sector([3, 3, -4, -4, -4, 5, -6, 7, -8, 9, 10, -11])[0].get('S')==1

In [16]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

CPU times: user 14.9 s, sys: 0 ns, total: 14.9 s
Wall time: 14.9 s


In [17]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [18]:
hs.shape #9466 solutions with hidden sector

(5196, 7)

In [19]:
hs.zmax.max()

20

In [20]:
hs[:1]

,l,k,solution,gcd,n,zmax,hidden
0,"[1, -1, 1]","[1, 0, -2]","[1, 1, 2, 3, -4, -4, -5, 6]",2,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1,..."


## Ask question to the solutions object

* Test the solution
* Which is the minimum hidden sector and with the least charge in absolute value
* Check for multigenerational solution
* Check for unconditional stability
* Solutions with active symmetry
* How many solutions with n=12 and zmax<=20
* Check for Majorana and Dirac fermions

Which `n` are allowed? 

In [21]:
hs['n'].unique()

array([ 8, 12])

In [22]:
pd.set_option('display.max_colwidth',200)

What is the minimum n?

In [23]:
hs['n'].min()

8

What is the minimum `z_max`?

In [24]:
hs['zmax'].min()

6

In [25]:
hs[hs['zmax']==hs['zmax'].min()]

,l,k,solution,gcd,n,zmax,hidden
0,"[1, -1, 1]","[1, 0, -2]","[1, 1, 2, 3, -4, -4, -5, 6]",2,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]"
292,"[1, -1, -2, -1, 1]","[1, 2, 0, -2, 0]","[1, 1, 1, 1, -2, -2, -2, 3, 3, -5, -5, 6]",4,12,6,"[{'S': 4, 'ψ': [(-2, -2), (1, -5), (-2, 6), (1, 3)]}]"
293,"[-2, 2, 1, -1, -2]","[1, 0, -1, 0, 1]","[1, -2, -2, -2, 3, 3, 4, 4, -5, -5, -5, 6]",2,12,6,"[{'S': 1, 'ψ': [(-5, 6), (-2, 3), (4, -5), (1, -2)]}, {'S': 7, 'ψ': [(3, 4), (1, 6), (-2, -5)]}]"


How many solutions have several dark sectors?

In [26]:
hs[hs['hidden'].apply(len)>1].shape

(1024, 7)

In [27]:
hs[hs['hidden'].apply(len)>2].shape

(0, 7)

In [28]:
hs['hidden'].str[0].str['ψ']

0                                      [(3, -5), (2, -4), (-4, 6), (1, 1)]
1                                      [(2, 2), (-3, 7), (1, -5), (2, -6)]
2                                               [(4, 5), (1, 8), (-2, -7)]
3                                      [(1, -6), (2, -7), (-3, 8), (1, 4)]
4                                      [(4, -7), (-5, 8), (2, -5), (1, 2)]
                                       ...                                
5191                             [(-16, 18), (18, -20), (-9, 11), (7, -9)]
5192                   [(-6, 7), (16, -17), (19, -20), (-17, 18), (1, -2)]
5193    [(-13, 14), (-16, 17), (19, -20), (-18, 19), (10, -11), (17, -18)]
5194                           [(-14, 17), (17, -20), (16, -19), (7, -10)]
5195                           [(9, -16), (-13, 20), (12, -19), (-10, 17)]
Name: hidden, Length: 5196, dtype: object

What is the minimum number of DM particles?

In [29]:
hs['hidden'].str[0].str['ψ'].apply(len).min()

3

What is the minimum number of DM particles?

In [30]:
hs['hidden'].str[-1].str['ψ'].apply(len).min()

3

In [31]:
hs.loc[[1773]]['hidden'].str[-1]

1773    {'S': 6, 'ψ': [(-10, 16), (4, -10), (11, -17), (2, -8), (2, 4), (-7, 13)]}
Name: hidden, dtype: object

In [32]:
hs.loc[[1773]]['hidden'].str[0]

1773    {'S': 6, 'ψ': [(-10, 16), (4, -10), (11, -17), (2, -8), (2, 4), (-7, 13)]}
Name: hidden, dtype: object

In [33]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==3)].shape

(108, 7)

to capture the cases with until with two dark sectors:

In [34]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==3) | 
   (hs['hidden'].str[-1].str['ψ'].apply(len)==3)]

,l,k,solution,gcd,n,zmax,hidden
2,"[-1, 2, -2]","[2, -1, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",5,8,8,"[{'S': 9, 'ψ': [(4, 5), (1, 8), (-2, -7)]}]"
7,"[-1, -3, -1]","[3, 2, 3]","[2, -3, -3, 5, 6, -8, -8, 9]",2,8,9,"[{'S': 11, 'ψ': [(5, 6), (-3, -8), (2, 9)]}]"
8,"[-3, 3, 2]","[-2, -3, 2]","[3, -4, -6, -6, 7, 7, 8, -9]",20,8,9,"[{'S': 1, 'ψ': [(8, -9), (-6, 7), (3, -4)]}]"
12,"[2, 0, -2]","[1, -1, 1]","[2, -5, -5, -5, 7, 8, 8, -10]",4,8,10,"[{'S': 3, 'ψ': [(7, -10), (-5, 8), (2, -5)]}]"
16,"[-2, 0, 2]","[-1, -3, 3]","[3, -4, -4, 6, 7, -9, -9, 10]",4,8,10,"[{'S': 13, 'ψ': [(3, 10), (-4, -9), (6, 7)]}]"
...,...,...,...,...,...,...,...
4528,"[-4, 3, -4, 3, 2]","[1, -2, 1, -2, 1]","[1, 1, 1, 2, 2, -5, 13, 13, -16, -16, -16, 20]",6,12,20,"[{'S': 15, 'ψ': [(1, -16), (-5, 20), (2, 13)]}]"
4888,"[-4, -2, -4, -2, -4]","[-3, 4, -3, 4, -3]","[15, -16, -16, -16, 17, 17, 18, 18, -19, -19, -19, 20]",56,12,20,"[{'S': 1, 'ψ': [(15, -16), (-16, 17), (18, -19), (-19, 20)]}, {'S': 35, 'ψ': [(-16, -19), (17, 18), (15, 20)]}]"
4952,"[-4, -2, -4, -3, -4]","[3, 4, 3, 4, 3]","[1, 1, -6, -6, -8, -8, -8, 13, 13, 13, 15, -20]",7,12,20,"[{'S': 5, 'ψ': [(1, -6), (-8, 13), (15, -20)]}, {'S': 7, 'ψ': [(-6, 13), (13, -20), (-8, 15), (1, -8)]}]"
5050,"[-4, 0, 4, 0, 4]","[-4, 2, -5, 4, -5]","[7, -8, -16, -16, -16, 17, 17, 17, 19, 19, -20, -20]",96,12,20,"[{'S': 1, 'ψ': [(7, -8), (-16, 17), (19, -20)]}]"


What is the maximum number of DM particles?

In [35]:
hs['hidden'].str[0].str['ψ'].apply(len).max()

9

What is the maximum number of DM particles?

In [36]:
hs['hidden'].str[-1].str['ψ'].apply(len).max()

9

In [37]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==9)][:1]

,l,k,solution,gcd,n,zmax,hidden
552,"[1, -2, 1, -2, 3]","[3, -3, 2, -2, 1]","[1, 1, 2, 3, -5, -6, -6, -7, 9, 10, 11, -13]",4,12,13,"[{'S': 4, 'ψ': [(9, -13), (-5, 9), (-6, 10), (2, 2), (1, 3), (-7, 11), (3, -7), (1, -5), (2, -6)]}, {'S': 12, 'ψ': [(-5, -7), (3, 9), (-6, -6), (1, -13), (1, 11), (2, 10)]}]"


What is the minimum number of _independent_ DM particles?

In [38]:
lst=[{'S': 3, 'ψ': [(-17, 20), (-23, 20), (14, -17)]}]
def intersections(lst,mask=True):
    lstc=lst.copy()
    lenmax=float('inf')
    for d in lst:
        for p in d.get('ψ'):
            dd=d.get('ψ').copy()
            dd.remove(p)
            [dd.remove(q) for q in dd.copy() if set(p).intersection(set(q))]
            if len(dd) < lenmax:
                ddd=dd.copy()
                lenmax=len(dd)
                
            if mask and not dd:
                return True
    if mask:        
        return False
    return ddd

assert intersections([{'S': 3, 'ψ': [(-17, 20), (-23, 20), (14, -17)]}])
assert not intersections([{'S': 3, 'ψ': [(-17, 20), (-7, 20), (14, -17), (2,5)]}])

One excluded:

In [39]:
hs[hs['hidden'].apply(intersections)]

,l,k,solution,gcd,n,zmax,hidden


The minimum is...

In [40]:
hs['~intersections']=hs['hidden'].apply(lambda lst: intersections(lst,mask=False))
hs[hs['~intersections'].apply(len)==hs['~intersections'].apply(len).min()].sort_values('n')

,l,k,solution,gcd,n,zmax,hidden,~intersections
4,"[2, -2, 1]","[2, 1, 2]","[1, 2, 2, 4, -5, -5, -7, 8]",4,8,8,"[{'S': 3, 'ψ': [(4, -7), (-5, 8), (2, -5), (1, 2)]}]","[(4, -7)]"
208,"[3, 0, 3]","[3, -3, 1]","[5, -9, -11, -11, 15, 15, 15, -19]",18,8,19,"[{'S': 4, 'ψ': [(-11, 15), (5, -9), (15, -19)]}]","[(5, -9)]"
154,"[-1, 1, -1]","[-5, -1, -5]","[1, 5, 7, 7, -11, -13, -13, 17]",6,8,17,"[{'S': 6, 'ψ': [(5, -11), (-11, 17), (1, 5), (7, -13)]}]","[(7, -13)]"
120,"[-2, 0, -2]","[-2, 2, -1]","[7, -10, -11, -11, 14, 14, 14, -17]",4,8,17,"[{'S': 3, 'ψ': [(-11, 14), (7, -10), (14, -17)]}]","[(7, -10)]"
103,"[-1, 0, 1]","[-1, 1, 4]","[1, 1, 1, -4, -11, 14, 14, -16]",1,8,16,"[{'S': 15, 'ψ': [(1, 14), (1, -16), (-4, -11)]}]","[(-4, -11)]"
67,"[2, 0, -2]","[2, -2, -3]","[1, -6, -6, -6, 9, 11, 11, -14]",4,8,14,"[{'S': 5, 'ψ': [(-6, 11), (9, -14), (1, -6)]}]","[(9, -14)]"
218,"[2, 4, -2]","[2, 3, 2]","[4, -9, -9, -11, 14, 14, 16, -19]",4,8,19,"[{'S': 5, 'ψ': [(-9, 14), (-11, 16), (14, -19), (4, -9)]}]","[(-11, 16)]"
39,"[1, 2, 1]","[1, -3, 1]","[1, -5, -5, -7, 9, 9, 11, -13]",2,8,13,"[{'S': 4, 'ψ': [(9, -13), (-5, 9), (-7, 11), (1, -5)]}]","[(-7, 11)]"
36,"[-2, 0, -2]","[-2, 2, 1]","[2, -5, -7, -7, 10, 10, 10, -13]",4,8,13,"[{'S': 3, 'ψ': [(-7, 10), (10, -13), (2, -5)]}]","[(2, -5)]"
35,"[-1, -2, -1]","[-1, 2, -1]","[4, -7, -7, -8, 10, 10, 11, -13]",1,8,13,"[{'S': 3, 'ψ': [(4, -7), (10, -13), (-7, 10), (-8, 11)]}]","[(-8, 11)]"


Babu analysis

In [41]:
hs8 = hs[hs.n == 8]
hs8.shape

(292, 8)

In [42]:
hs.hidden.str[0].str['S'].value_counts()[:4]

hidden
1    912
3    727
2    569
5    501
Name: count, dtype: int64

In [43]:
hs[hs.hidden.str[0].str['S'] == 2][:20]

,l,k,solution,gcd,n,zmax,hidden,~intersections
0,"[1, -1, 1]","[1, 0, -2]","[1, 1, 2, 3, -4, -4, -5, 6]",2,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]","[(3, -5), (1, 1)]"
15,"[1, 2, 1]","[2, -3, 2]","[1, -3, -5, -6, 7, 8, 8, -10]",4,8,10,"[{'S': 2, 'ψ': [(1, -3), (8, -10), (-5, 7), (-6, 8), (1, 1)]}]","[(8, -10), (-5, 7), (-6, 8)]"
29,"[3, 2, -1]","[-2, 3, -1]","[1, -3, -4, 6, -9, 10, 11, -12]",9,8,12,"[{'S': 2, 'ψ': [(10, -12), (1, -3), (-9, 11), (-4, 6), (1, 1)]}]","[(10, -12), (-9, 11), (-4, 6)]"
50,"[1, -1, -2]","[2, 0, 2]","[1, 1, 4, -6, 11, -12, -13, 14]",2,8,14,"[{'S': 2, 'ψ': [(4, -6), (11, -13), (-12, 14), (1, 1)]}]","[(11, -13), (-12, 14), (1, 1)]"
59,"[-1, -3, 2]","[-2, -3, -2]","[3, -5, -7, 9, -10, 12, 12, -14]",4,8,14,"[{'S': 2, 'ψ': [(3, -5), (-10, 12), (-7, 9), (12, -14)]}]","[(3, -5), (-7, 9)]"
74,"[-2, 1, -1]","[-3, 2, 1]","[4, -6, -9, -10, 11, 12, 13, -15]",4,8,15,"[{'S': 2, 'ψ': [(4, -6), (-9, 11), (-10, 12), (13, -15)]}]","[(-9, 11), (-10, 12), (13, -15)]"
79,"[1, -1, 1]","[-3, -1, 3]","[1, -3, -9, -9, 11, 11, 13, -15]",2,8,15,"[{'S': 2, 'ψ': [(1, -3), (-9, 11), (13, -15), (1, 1)]}]","[(-9, 11), (13, -15)]"
80,"[-1, -2, -1]","[-3, 0, 2]","[2, -4, -5, 7, -12, 13, 14, -15]",2,8,15,"[{'S': 2, 'ψ': [(2, -4), (-12, 14), (13, -15), (-5, 7)]}]","[(-12, 14), (13, -15), (-5, 7)]"
107,"[-1, 1, -1]","[-1, 2, 4]","[1, 1, 8, -10, 11, -13, -14, 16]",2,8,16,"[{'S': 2, 'ψ': [(11, -13), (8, -10), (1, 1), (-14, 16)]}]","[(8, -10), (1, 1), (-14, 16)]"
122,"[2, 1, 3]","[-3, -2, 1]","[2, -4, -10, -11, 12, 13, 15, -17]",4,8,17,"[{'S': 2, 'ψ': [(2, -4), (-10, 12), (-11, 13), (15, -17)]}]","[(-10, 12), (-11, 13), (15, -17)]"


In [44]:
hs8[hs8.solution.apply(lambda L: [x for x in L if x%2 == 0]).apply(len) == 0]

,l,k,solution,gcd,n,zmax,hidden,~intersections
9,"[1, -1, -3]","[3, 1, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",4,8,9,"[{'S': 4, 'ψ': [(1, -5), (-5, 9), (3, -7), (1, 3)]}]","[(3, -7)]"
39,"[1, 2, 1]","[1, -3, 1]","[1, -5, -5, -7, 9, 9, 11, -13]",2,8,13,"[{'S': 4, 'ψ': [(9, -13), (-5, 9), (-7, 11), (1, -5)]}]","[(-7, 11)]"
42,"[1, -2, 3]","[1, -3, 1]","[1, 1, 1, -5, -7, 11, 11, -13]",6,8,13,"[{'S': 12, 'ψ': [(1, 11), (-5, -7), (1, -13)]}]","[(-5, -7)]"
43,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 6, 'ψ': [(5, -11), (3, -9), (3, 3), (-7, 13)]}]","[(5, -11), (-7, 13)]"
79,"[1, -1, 1]","[-3, -1, 3]","[1, -3, -9, -9, 11, 11, 13, -15]",2,8,15,"[{'S': 2, 'ψ': [(1, -3), (-9, 11), (13, -15), (1, 1)]}]","[(-9, 11), (13, -15)]"
85,"[-3, 1, -1]","[1, -3, 1]","[1, -3, -3, 7, 9, -13, -13, 15]",10,8,15,"[{'S': 16, 'ψ': [(1, 15), (-3, -13), (7, 9)]}]","[(-3, -13), (7, 9)]"
123,"[3, -1, -2]","[3, -1, 1]","[1, -3, -7, 9, -13, 15, 15, -17]",10,8,17,"[{'S': 2, 'ψ': [(1, -3), (15, -17), (-13, 15), (1, 1), (-7, 9)]}]","[(15, -17), (-13, 15), (-7, 9)]"
127,"[1, 2, 3]","[1, -3, 1]","[1, 3, 3, -5, 9, -13, -15, 17]",2,8,17,"[{'S': 12, 'ψ': [(3, -15), (3, 9), (1, -13), (-5, 17)]}]","[(1, -13), (-5, 17)]"
130,"[-1, 0, -3]","[-3, 1, -3]","[3, -5, -5, 9, 11, -15, -15, 17]",8,8,17,"[{'S': 20, 'ψ': [(-5, -15), (9, 11), (3, 17)]}]","[(9, 11), (3, 17)]"
132,"[-3, -1, -3]","[1, 3, -1]","[3, 3, 5, 7, -9, -11, -15, 17]",10,8,17,"[{'S': 8, 'ψ': [(-9, 17), (7, -15), (3, 5), (3, -11)]}]","[(-9, 17), (7, -15)]"


In [45]:
asfad

NameError: name 'asfad' is not defined

### Appendix: Same questions for Standard Model (SM) solutions
Which are solutions with three repeated integers to be asigned to SM particles

In [46]:
df3=df[df['solution'].apply(lambda l: 3 in [l.count(x) for x in l])].reset_index(drop=True)

Remove the three repeated integers from the new `hidden` column

In [47]:
def get_dark(l,idx=0):
    #TODO,
    xxx=list(set([x for x in l if l.count(x)==3]))
    if len(xxx)>=idx+1:
        xx=xxx[idx]
        return [x for x in l if x!=xx]
    else:
        return []
assert get_dark([1,1,2,2,2,3] )==[1, 1, 3]
assert get_dark([2,2,2,2,-5,-5,-5,7])==[2, 2, 2, 2, 7]

In [48]:
df3['hidden']=df3['solution'].apply(get_dark).apply(get_hidden_sector)

In [49]:
hs3=df3[df3['hidden'].apply(len)>0].reset_index(drop=True)
hs3.shape

(49, 7)

In [50]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=1)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(51, 7)

In [51]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=2)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(51, 7)

In [52]:
hs3['zmax'].min()

9

In [53]:
hs3[:1]

,l,k,solution,gcd,n,zmax,hidden
0,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]"


In [54]:
hs3[hs3['hidden'].apply(len)>1].shape

(0, 7)

In [55]:
hs3[hs3['hidden'].apply(len)>2].shape

(0, 7)

In [56]:
hs3[hs3['hidden'].apply(len)>3].shape

(0, 7)

In [57]:
hs3['hidden'].str[0].str['ψ'].apply(len).min()

2

In [58]:
hs3[hs3['hidden'].str[0].str['ψ'].apply(len)==hs3['hidden'].str[0].str['ψ'].apply(len).min()]

,l,k,solution,gcd,n,zmax,hidden
2,"[-2, 0, 2]","[-2, 1, 0, -2]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",4,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]"
3,"[1, 0, -1]","[-3, 3, 0, -3]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",9,9,12,"[{'S': 11, 'ψ': [(1, -12), (-3, -8)]}]"
5,"[3, 0, -3]","[-2, 2, 0, -2]","[4, -5, -5, -6, -6, 11, 11, 11, -15]",8,9,15,"[{'S': 11, 'ψ': [(-5, -6), (4, -15)]}]"
12,"[2, 0, -2]","[2, -2, 0, -3]","[2, 2, -3, 15, 15, -17, -17, -17, 20]",4,9,20,"[{'S': 17, 'ψ': [(-3, 20), (2, 15)]}]"


Check if `sum([L,L,L,x,y])` where `(x,y) = ±S`  

In [59]:
def check_L(r):
    L=[x for x in r.get('solution') if r.get('solution').count(x)==3 ]
    for LL in L[::3]:
        z=r.get('hidden')[0]
        for p in z.get('ψ'):
            if 3*LL+sum(list(p))==0:
                return list(p)
    return []
assert check_L({'solution':[4,4,4,5,-8,-10,13,-9, -9, -9, 15, 15, 15],
         'hidden':[{'S': 18, 'ψ': [(-9, -9), (-8, -10), (5, 13)]}]})==[]

In [60]:
hs3[hs3.apply(check_L,axis='columns').apply(len)>0].reset_index(drop=True)

,l,k,solution,gcd,n,zmax,hidden
0,"[-2, -3, 0]","[1, 2, 3, 2]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",4,9,13,"[{'S': 18, 'ψ': [(-9, -9), (-8, -10), (5, 13)]}]"
1,"[2, 0, 3]","[-2, 1, -3, -1]","[2, 3, 3, 3, 6, -8, -11, -15, 17]",12,9,17,"[{'S': 9, 'ψ': [(2, -11), (6, -15), (-8, 17)]}]"
2,"[-4, -2, 1]","[2, -4, 4, -2]","[1, -2, 6, 6, 6, -9, -9, -16, 17]",16,9,17,"[{'S': 18, 'ψ': [(-9, -9), (1, 17), (-2, -16)]}]"
3,"[3, -2, -4]","[-2, -1, -2, 3]","[1, 2, 3, -6, -6, -6, 15, 16, -19]",2,9,19,"[{'S': 18, 'ψ': [(1, -19), (3, 15), (2, 16)]}]"
4,"[6, -3, 1]","[-1, -3, -6, -3]","[7, 9, 9, 9, -11, -12, -15, -16, 20]",18,9,20,"[{'S': 27, 'ψ': [(7, 20), (-12, -15), (-11, -16)]}]"


### Extend the data scheme with number of generations

In [61]:
def get_Dirac_and_Majorana_generations(l,h):
    g=[]
    dgmax=0
    mgmax=0
    for idx in range(len(h)):
        for x,y in h[idx].get('ψ'):
            if x!=y and l.count(x)>dgmax and l.count(y)==l.count(x):
                    dgmax=l.count(x)
            if x==y and l.count(x)>mgmax:
                    mgmax=l.count(x)
        g.append({'DG':dgmax,'MG':mgmax})
    return g
l=[2, 2, -3, 15, 15, 20]
h=[{'S': 17, 'ψ': [(2, 15), (20, -3)]}]
assert get_Dirac_and_Majorana_generations(l,h)==[{'DG': 2, 'MG': 0}]

In [62]:
hs[:3].apply(lambda d: d['l']+d['k'],axis='columns')

0     [1, -1, 1, 1, 0, -2]
1     [1, -2, 2, 2, -1, 0]
2    [-1, 2, -2, 2, -1, 2]
dtype: object

In [63]:
hs['Generations']=hs.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [64]:
hs3['Generations']=hs3.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [65]:
hs['Gmax']=hs['Generations'].str[0].apply(lambda d: sum(d.values()))

In [66]:
hs=hs.sort_values(['Gmax'],ascending=False).reset_index(drop=True)

In [67]:
hs[hs['n']==8]

,l,k,solution,gcd,n,zmax,hidden,~intersections,Generations,Gmax
192,"[1, -2, 2]","[2, -1, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(2, 2), (-3, 7), (1, -5), (2, -6)]}]","[(-3, 7), (1, -5)]","[{'DG': 1, 'MG': 3}]",4
248,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 6, 'ψ': [(5, -11), (3, -9), (3, 3), (-7, 13)]}]","[(5, -11), (-7, 13)]","[{'DG': 1, 'MG': 3}]",4
255,"[-3, 0, -1]","[1, 3, 1]","[3, 5, 5, 7, -9, -15, -15, 19]",4,8,19,"[{'S': 10, 'ψ': [(3, 7), (5, -15), (-9, 19), (5, 5)]}]","[(3, 7), (-9, 19)]","[{'DG': 2, 'MG': 2}]",4
319,"[-1, 0, -5]","[3, 1, -1]","[1, -6, -6, -6, 7, 11, 18, -19]",8,8,19,"[{'S': 12, 'ψ': [(-6, -6), (-6, 18), (1, 11), (7, -19)]}]","[(1, 11), (7, -19)]","[{'DG': 1, 'MG': 3}]",4
626,"[1, -1, 1]","[1, 0, -2]","[1, 1, 2, 3, -4, -4, -5, 6]",2,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]","[(3, -5), (1, 1)]","[{'DG': 1, 'MG': 2}]",3
...,...,...,...,...,...,...,...,...,...,...
3710,"[-3, -4, -3]","[2, -2, -1]","[1, 3, 8, -12, 13, -16, -17, 20]",4,8,20,"[{'S': 4, 'ψ': [(8, -12), (1, 3), (-16, 20), (13, -17)]}]","[(1, 3), (-16, 20), (13, -17)]","[{'DG': 1, 'MG': 0}]",1
3712,"[1, 4, -3]","[2, 3, 2]","[4, -7, -10, 13, -14, 17, 17, -20]",6,8,20,"[{'S': 3, 'ψ': [(-10, 13), (4, -7), (-14, 17), (17, -20)]}]","[(-10, 13), (4, -7)]","[{'DG': 1, 'MG': 0}]",1
3715,"[4, 2, -4]","[1, 2, -3]","[1, 4, 5, -7, 9, -14, -18, 20]",4,8,20,"[{'S': 13, 'ψ': [(4, 9), (5, -18), (-7, 20), (1, -14)]}]","[(5, -18), (-7, 20), (1, -14)]","[{'DG': 1, 'MG': 0}]",1
3716,"[4, 2, 4]","[-3, -1, 1]","[1, -2, -6, 9, 12, -15, -19, 20]",8,8,20,"[{'S': 21, 'ψ': [(-6, -15), (1, 20), (9, 12), (-2, -19)]}]","[(1, 20), (9, 12), (-2, -19)]","[{'DG': 1, 'MG': 0}]",1


In [ ]:
hs[hs['n']==8][:6]

In [ ]:
hs[hs['n']==9][:3]

In [ ]:
hs[hs['n']==11][:4]

In [ ]:
hs[hs['n']==12][:4]

In [68]:
hs3['Gmax']=-hs3['Generations'].str[0].apply(lambda d: sum(d.values()))
hs3=hs3.sort_values(['n','Gmax']).reset_index(drop=True)

In [69]:
hs3[hs3['n']==8][:1]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]","[{'DG': 1, 'MG': 1}]",-2


In [70]:
hs3[hs3['n']==9][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1,"[-2, -3, 0]","[1, 2, 3, 2]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",4,9,13,"[{'S': 18, 'ψ': [(-9, -9), (-8, -10), (5, 13)]}]","[{'DG': 1, 'MG': 2}]",-3
2,"[-4, -2, 1]","[2, -4, 4, -2]","[1, -2, 6, 6, 6, -9, -9, -16, 17]",16,9,17,"[{'S': 18, 'ψ': [(-9, -9), (1, 17), (-2, -16)]}]","[{'DG': 1, 'MG': 2}]",-3
3,"[-2, 0, 2]","[-2, 1, 0, -2]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",4,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]","[{'DG': 2, 'MG': 0}]",-2
4,"[1, 0, -1]","[-3, 3, 0, -3]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",9,9,12,"[{'S': 11, 'ψ': [(1, -12), (-3, -8)]}]","[{'DG': 2, 'MG': 0}]",-2


In [71]:
hs3[hs3['n']==10][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
15,"[-3, -1, -3, -1]","[-3, 0, 2, -1]","[3, 3, 3, 4, -6, -7, 9, -11, -12, 14]",6,10,14,"[{'S': 18, 'ψ': [(-6, -12), (4, 14), (-7, -11), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2
16,"[-1, 0, 2, -1]","[3, -3, 3, -1]","[1, -5, -7, 9, -11, -13, 15, 15, 15, -19]",4,10,19,"[{'S': 18, 'ψ': [(-5, -13), (1, -19), (-7, -11), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2
17,"[-2, 1, 0, 3]","[3, 0, -3, 1]","[1, -2, 3, 3, 3, -6, 9, -12, -19, 20]",6,10,20,"[{'S': 18, 'ψ': [(-2, 20), (-6, -12), (1, -19), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2


In [72]:
hs3[hs3['n']==11][:2]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
18,"[-4, 0, 2, 0]","[-4, 4, -4, 3, -3]","[4, 5, 8, -9, -9, 10, -12, -12, -12, 13, 14]",64,11,14,"[{'S': 18, 'ψ': [(5, 13), (4, 14), (-9, -9), (8, 10)]}]","[{'DG': 1, 'MG': 2}]",-3
19,"[-2, -3, 0, 3]","[-3, -1, -2, 3, -1]","[1, 1, -2, 3, -6, -6, -6, 8, 8, 11, -12]",15,11,12,"[{'S': 9, 'ψ': [(-2, 11), (3, -12), (1, 8)]}]","[{'DG': 2, 'MG': 0}]",-2


In [73]:
hs3[hs3['n']==12][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
33,"[-1, -6, -5, 1, 7]","[-4, -7, -2, 7, 4]","[2, 8, -9, -9, -9, -9, -9, 10, 15, 15, 15, -20]",126,12,20,"[{'S': 18, 'ψ': [(-9, -9), (8, 10), (2, -20)]}]","[{'DG': 1, 'MG': 5}]",-6
34,"[-1, 3, 2, 3, -1]","[3, -3, -1, -3, -1]","[1, 1, 3, 3, 3, 5, 5, -7, -9, -9, -9, 13]",8,12,13,"[{'S': 6, 'ψ': [(1, 5), (3, 3), (-7, 13), (1, -7)]}]","[{'DG': 2, 'MG': 3}]",-5
35,"[1, 3, 2, 0, -3]","[-3, -2, -1, -2, -3]","[3, 3, 3, 6, 7, -8, -9, -9, -9, -10, 11, 12]",8,12,12,"[{'S': 18, 'ψ': [(6, 12), (-9, -9), (-8, -10), (7, 11)]}]","[{'DG': 1, 'MG': 3}]",-4
36,"[2, 0, 2, 4, -2]","[2, 5, 4, -4, -5]","[2, 2, 4, -5, 6, -12, -12, -14, 15, 15, 15, -16]",24,12,16,"[{'S': 10, 'ψ': [(4, 6), (6, -16), (4, -14), (-5, -5), (2, -12)]}]","[{'DG': 2, 'MG': 1}]",-3
37,"[3, 2, 0, -1, 1]","[3, 1, 2, 3, -2]","[1, 1, -6, 8, 9, 10, -12, -15, -15, -15, 17, 17]",4,12,17,"[{'S': 18, 'ψ': [(-6, -12), (1, 17), (9, 9), (8, 10)]}]","[{'DG': 2, 'MG': 1}]",-3


In [74]:
hs3['~intersections']=hs3['hidden'].apply(lambda lst: intersections(lst,mask=False))
hs3[hs3['~intersections'].apply(len)==hs3['~intersections'].apply(len).min()].sort_values('n')

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax,~intersections
3,"[-2, 0, 2]","[-2, 1, 0, -2]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",4,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]","[{'DG': 2, 'MG': 0}]",-2,"[(-4, -5)]"
4,"[1, 0, -1]","[-3, 3, 0, -3]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",9,9,12,"[{'S': 11, 'ψ': [(1, -12), (-3, -8)]}]","[{'DG': 2, 'MG': 0}]",-2,"[(-3, -8)]"
5,"[3, 0, -3]","[-2, 2, 0, -2]","[4, -5, -5, -6, -6, 11, 11, 11, -15]",8,9,15,"[{'S': 11, 'ψ': [(-5, -6), (4, -15)]}]","[{'DG': 2, 'MG': 0}]",-2,"[(4, -15)]"
6,"[2, 0, -2]","[2, -2, 0, -3]","[2, 2, -3, 15, 15, -17, -17, -17, 20]",4,9,20,"[{'S': 17, 'ψ': [(-3, 20), (2, 15)]}]","[{'DG': 2, 'MG': 0}]",-2,"[(2, 15)]"
34,"[-1, 3, 2, 3, -1]","[3, -3, -1, -3, -1]","[1, 1, 3, 3, 3, 5, 5, -7, -9, -9, -9, 13]",8,12,13,"[{'S': 6, 'ψ': [(1, 5), (3, 3), (-7, 13), (1, -7)]}]","[{'DG': 2, 'MG': 3}]",-5,"[(3, 3)]"


In [78]:
hs3[(hs3.Generations.str[0].str['MG']==2) | (hs3.Generations.str[0].str['MG']==3)]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax,~intersections
1,"[-2, -3, 0]","[1, 2, 3, 2]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",4,9,13,"[{'S': 18, 'ψ': [(-9, -9), (-8, -10), (5, 13)]}]","[{'DG': 1, 'MG': 2}]",-3,"[(-8, -10), (5, 13)]"
2,"[-4, -2, 1]","[2, -4, 4, -2]","[1, -2, 6, 6, 6, -9, -9, -16, 17]",16,9,17,"[{'S': 18, 'ψ': [(-9, -9), (1, 17), (-2, -16)]}]","[{'DG': 1, 'MG': 2}]",-3,"[(1, 17), (-2, -16)]"
18,"[-4, 0, 2, 0]","[-4, 4, -4, 3, -3]","[4, 5, 8, -9, -9, 10, -12, -12, -12, 13, 14]",64,11,14,"[{'S': 18, 'ψ': [(5, 13), (4, 14), (-9, -9), (8, 10)]}]","[{'DG': 1, 'MG': 2}]",-3,"[(4, 14), (-9, -9), (8, 10)]"
34,"[-1, 3, 2, 3, -1]","[3, -3, -1, -3, -1]","[1, 1, 3, 3, 3, 5, 5, -7, -9, -9, -9, 13]",8,12,13,"[{'S': 6, 'ψ': [(1, 5), (3, 3), (-7, 13), (1, -7)]}]","[{'DG': 2, 'MG': 3}]",-5,"[(3, 3)]"
35,"[1, 3, 2, 0, -3]","[-3, -2, -1, -2, -3]","[3, 3, 3, 6, 7, -8, -9, -9, -9, -10, 11, 12]",8,12,12,"[{'S': 18, 'ψ': [(6, 12), (-9, -9), (-8, -10), (7, 11)]}]","[{'DG': 1, 'MG': 3}]",-4,"[(-9, -9), (-8, -10), (7, 11)]"


In [79]:
hs3[(hs3.Generations.str[0].str['MG']==1) ]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax,~intersections
0,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]","[{'DG': 1, 'MG': 1}]",-2,"[(-9, -9), (5, 13)]"
15,"[-3, -1, -3, -1]","[-3, 0, 2, -1]","[3, 3, 3, 4, -6, -7, 9, -11, -12, 14]",6,10,14,"[{'S': 18, 'ψ': [(-6, -12), (4, 14), (-7, -11), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2,"[(4, 14), (-7, -11), (9, 9)]"
16,"[-1, 0, 2, -1]","[3, -3, 3, -1]","[1, -5, -7, 9, -11, -13, 15, 15, 15, -19]",4,10,19,"[{'S': 18, 'ψ': [(-5, -13), (1, -19), (-7, -11), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2,"[(1, -19), (-7, -11), (9, 9)]"
17,"[-2, 1, 0, 3]","[3, 0, -3, 1]","[1, -2, 3, 3, 3, -6, 9, -12, -19, 20]",6,10,20,"[{'S': 18, 'ψ': [(-2, 20), (-6, -12), (1, -19), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2,"[(-6, -12), (1, -19), (9, 9)]"
36,"[2, 0, 2, 4, -2]","[2, 5, 4, -4, -5]","[2, 2, 4, -5, 6, -12, -12, -14, 15, 15, 15, -16]",24,12,16,"[{'S': 10, 'ψ': [(4, 6), (6, -16), (4, -14), (-5, -5), (2, -12)]}]","[{'DG': 2, 'MG': 1}]",-3,"[(-5, -5), (2, -12)]"
37,"[3, 2, 0, -1, 1]","[3, 1, 2, 3, -2]","[1, 1, -6, 8, 9, 10, -12, -15, -15, -15, 17, 17]",4,12,17,"[{'S': 18, 'ψ': [(-6, -12), (1, 17), (9, 9), (8, 10)]}]","[{'DG': 2, 'MG': 1}]",-3,"[(1, 17), (9, 9), (8, 10)]"
38,"[-5, 3, 0, 5, -4]","[-5, 4, 2, -4, 2]","[2, 6, -7, -7, -9, -11, -11, 12, 15, 15, 15, -20]",135,12,20,"[{'S': 18, 'ψ': [(6, 12), (2, -20), (-7, -11), (-9, -9)]}]","[{'DG': 2, 'MG': 1}]",-3,"[(2, -20), (-7, -11), (-9, -9)]"
39,"[1, 0, -2, 1, -2]","[-3, -1, 1, -2, 0]","[3, 3, 3, 4, -5, 7, -8, -9, -10, 11, -13, 14]",4,12,14,"[{'S': 18, 'ψ': [(-5, -13), (4, 14), (-9, -9), (-8, -10), (7, 11)]}]","[{'DG': 1, 'MG': 1}]",-2,"[(4, 14), (-9, -9), (-8, -10), (7, 11)]"
40,"[-1, -2, -1, -2, 3]","[3, 4, -2, -3, 3]","[2, 3, 3, 3, -4, -5, 7, -9, 11, -13, -14, 16]",6,12,16,"[{'S': 18, 'ψ': [(-4, -14), (-5, -13), (2, 16), (-9, -9), (7, 11)]}]","[{'DG': 1, 'MG': 1}]",-2,"[(-5, -13), (2, 16), (-9, -9), (7, 11)]"
41,"[1, 3, -2, -3, 3]","[-3, 0, -3, -1, 0]","[1, 2, 5, -6, 9, -12, 13, -15, -15, -15, 16, 17]",8,12,17,"[{'S': 18, 'ψ': [(5, 13), (-6, -12), (2, 16), (1, 17), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2,"[(-6, -12), (2, 16), (1, 17), (9, 9)]"


## Appendix: Use merge to compare two DataFrames

In [ ]:
df=pd.read_json('solutions.json.gz')

In [ ]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

In [ ]:
hs2=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()
hs2.shape

Columns can only be compared if they are simple objects, like strings:

In [ ]:
hs['ss']  = hs['solution'].apply(str)
hs2['ss'] = hs2['solution'].apply(str)

Increase the with of the columns for easier visualization.

In [ ]:
pd.set_option('display.max_colwidth',200)

Compare the larger DataFrame (with filtered columns) against the smaller. The non `on='s'` repeated columns will appears with suffixes. 

In [ ]:
tmp=hs2[['ss','hidden']].merge(hs[['ss','hidden']],on='ss',how='left',suffixes=('_1','_2'))

In [ ]:
#tmp=
tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )].reset_index(drop=True)

In [ ]:
tmp['zmax']=tmp['ss'].apply(eval).str[-1].abs()
tmp['n']=tmp['ss'].apply(eval).apply(len)

In [ ]:
tmp=tmp.sort_values(['n','zmax'])

There are plenty of new solutions with the new dataset build with the Multiprocessing algorithm in the previous Chapter

In [ ]:
tmp

In [ ]:
hs.shape

In [ ]:
tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )][:5][["ss",'hidden_1']]

In [ ]:
import itertools
import math as mh

def remove_double_intersection_pairs(msvp,l):
    '''
    Example:
        msvp = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        # Remove (1,6) because is fully contained 
        # in the intersections with (_1_,-8) and (_6_,-13)
        returns: 
        [(3,4),(6,-13),(-7,14),(1,-8)]    
    '''
    for p in msvp.copy():
        tmp=msvp.copy()
        tmp.remove(p)
        fltmp=set([x for sublist in tmp for x in sublist]) 
        if l.count(p[0])==1 and l.count(p[1])==1 and set(p).issubset(fltmp):
            msvp=tmp.copy()
    return msvp
assert len(remove_double_intersection_pairs([(-10, 11), (-12, 13), (11, -12)],
                                             [-10, 11, -12, 13]))==2
assert len(remove_double_intersection_pairs([(4,4),(4,-12)],[4,-12]))==1

def get_real_massless(l,msvp):
    '''
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        msvp = [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        [] # lp    
    '''
    lp=l.copy()
    for x,y in msvp:
        #print(lp,msvp,x,y)
        if lp and x in lp and y in lp:
            minxy=min(lp.count(x),lp.count(y))
            if x!=y:
                [(lp.remove(x),lp.remove(y)) for ii in range(minxy)]
            else:
                [ lp.remove(x) for ii in range(lp.count(x)) ] # abs(2*x)==s
        else: # not lp
            break
    return lp

def get_hidden_fermions(l,s,ps=[]):
    '''
    For a given scalar Abelian charge `s`, build the 
    fermion non-zero determinant mass matrix from their
    Abelian charges in `l`, and returns the input `s` and 
    the non-zero mass entries.
    Returns an empty dictionary otherwise.
    
    The list of combinations on pairs of `l` can be given in `ps`,
    otherwise it is internally calculated.
    
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        s = 7
        ps = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        # If necessary temove [1,6] because have is fully contained 
        # in the intersections with others
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        {'s':7, '\psi':[(3,4),(6,-13),(1,6),(-7,14),(1,-8)]}

    '''
    sltn={}
    if not ps:
        ps = list(set(itertools.combinations_with_replacement(l,2)))
    msv = [p for p in ps if abs(sum(p))==s]
    lmsv=[x for sublist in msv for x in sublist]
    massives=set(lmsv)        
    massless=set(l).difference(massives)
    if not massless:
        sltn['S']=s
        sltn['ψ']=msv
    return sltn
assert get_hidden_fermions([2, 4, 4, 5, 5, 7],9).get('S')==9    


def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        sltn=get_hidden_fermions(l,s,ps)
        if sltn:
            sltns.append(sltn)
    return sltns



In [ ]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

In [ ]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [ ]:
hs.shape #9466 solutions with hidden sector

In [ ]:
hs.solution.str[-1].abs()

In [ ]:
hs8=hs[ (hs.n==8) & (hs.solution.str[-1].abs()<20)][['solution','hidden']]

In [ ]:
hs8['hidden']=hs8['hidden'].str[0]

In [ ]:
hs8=hs8.rename({'solution':'lista','hidden':'pares'},axis='columns')

In [ ]:
hs8

In [ ]:
hs8['pares']=hs8.pares.apply(lambda d: {'S':d.get('S'),'ψ':d.get('ψ')+[(y,x) for x,y in d.get('ψ') if x!=y]})

In [ ]:
hs8.to_json('lista_pares.json',orient='records')

In [ ]:
df=pd.read_json('lista_pares.json')

In [ ]:
df.iloc[1]

In [ ]:
import numpy as np

A) Cargue los datos disponibles en:

[https://github.com/ComputationalMethods/Evaluacion_2022-1/blob/main/data/lista_pares.json](https://raw.githubusercontent.com/ComputationalMethods/Evaluacion_2022-1/main/data/lista_pares.json)

como un DataFrame de pandas. Este contendrá dos columnas. La columna `lista`, con una lista de 8 enteros (algunos de ellos repetidos) y una columna `pares` correspondiente a un diccionario
con dos items: un entero rotulado con `'S'` y una lista de pares de enteros routulada con `'ψ'`. Note que el resultado de la suma en valor absoluto de los dos enteros en cada uno de los pares reproduce el valor del correspondiente `'S'` 

Seleccione la fila del DataFrame con el índice correspondiente a los dos últimos dígitos de su cédula y desarrolle los siguientes puntos con la correspondiente  `lista` y `pares`:
1. Construya una matrix $8\times 8$ _simétrica_ donde cada fila y la correspondiente columna correspondan a un entero de la lista (incluyendo los repetidos); de manera que en los cruces de filas y columnas definidos por los pares de enteros, se genere una entrada aleatoria mayor que cero y menor o igual que 1, con las demas entradas iguales a cero.
1. Calcule el determinante y establezca explícitamente si es diferente cero (tenga en cuenta la precisión numérica). Recuerde que si el determinante es cero, se espera que al menos uno de los autovalores sea cero
2. Organize los autovalores y autovectores de menor a mayor en valor absoluto y compruebe que las matriz de autovectores organizada da lugar a los autovalores en el orden correcto. 
3. Establezca explícitamente _cuantos_ autovalores son degenerados en valor absoluto.

```
   1 2 2 2 -3 -5 -6 7
1  0 0 0 0  0  x  0 0
2  0 x x x  0  0  x 0
2  0 x x x  0  0  x 0
2  0 x x x  0  0  x 0
-3 0 0 0 0  0  0  0 x
-5 x 0 0 0  0  0  0 0
-6 0 x x x  0  0  0 0
7  0 0 0 0  x  0  0 0
```

https://stackoverflow.com/questions/65862611/integration-of-sin-x-dx-problem-in-python-programming-implementation

In [ ]:
from scipy import integrate

B)
1. Usando su número de cédula cómo semilla, obtenga un numero entero aleatorio, $n$, entre 2 y 20.
2. Haga una estimación de la siguiente integral hasta dos cifras significativas, usando el método `quad` del módulo `integrate` de Scipy
$$
\int_0^\infty \sin(x^n)\operatorname{d}x
$$
Sugerencia. Compare con el valor obtenido desde https://www.wolframalpha.com/ o [sympy](https://stackoverflow.com/questions/65862611/integration-of-sin-x-dx-problem-in-python-programming-implementation)

In [ ]:
integrate.quad(lambda x: np.sin(x**20),0,1.5,limit=5000)

In [ ]:
A=np.random.random((8,8)).round(2)
A=(A+A.transpose())/2
A[0,0:5]=0;A[0,6:]=0;
A[1:4,0]=0; A[1:4,4:6]=0;A[1:4,7]=0
#A[1:4,0]=0; A[1:4,2:6]=0;A[1:4,7]=0
#A[1:4,0]=0; A[1:4,2:6]=0;A[1:4,7]=0
A[4,0:7]=0
A[5,1:8]=0;
A[6,0]=0;A[6,4:]=0;
A[7,0:4]=0;A[7,5:]=0;

print(A)

In [ ]:
np.linalg.det(A)

In [ ]:
np.linalg.eig(A)

In [ ]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

In [ ]:
df['solution']==